# Examples 2: Annuity with DAV Annuity Table

In the current example we build on the previous one and demonstrate two additional things:

  * using the DAV2004R table as the base mortality assumption
  * starting the projector from code
 
 
 The DAV2004R table can be used in a variety of ways for different use cases and *protolinc* provides
 some convenience code to use some of those with little effort.

### Download the DAV Tables

It's not so clear why the DAV tables aren't easily available for downloading from official sources. Luckily, the maintainer of the R-Project *MortalityTables* (cf. https://www.rdocumentation.org/packages/MortalityTables/versions/2.0.3) provides the required files. If you are willing to get them (the legal situation around this is not clear to the author) open up a command prompt and change into the base directory of this example and run:
```
protolinc download_dav_files
```
**or** run the folling code from a script:

In [1]:
from protolinc.utils import download_dav_tables
download_dav_tables()

Now a new folder called *tables* should have been created in  the working folder with two subfolders. In the present case the one of interest is this:

In [2]:
# use when not on Windows "!ls"
!dir /B tables\Germany_Annuities_DAV2004R

Germany_Annuities_DAV2004R.csv
Germany_Annuities_DAV2004R_AV.csv
Germany_Annuities_DAV2004R_AVBase.csv
Germany_Annuities_DAV2004R_Select.csv
Germany_Annuities_DAV2004R_Trends.csv


*Protolinc* provides an interface class to these files. On instantiation pass the base folder from which to load the file as an argument.

In [3]:
from protolinc.assumptions.dav2004r import DAV2004R
dav2004r = DAV2004R(base_directory="tables/Germany_Annuities_DAV2004R", trend_t1=10, trend_t2=25)

The second and third argument above control the trend factor extrapolation between the start and target trends.


As an aside it is quickly demonstrated here how one can work with this *raw* object (i.e. outside the context of a projection run). In this case one first has to create a *rates provider* object. Specifically, in this case we parametrize the *aggregate best estimate rates*, i.e. the aggregate (non-select) version using no additional prudency loadings ("2. Ordnung").

In [4]:
rates_provider = dav2004r.rates_provider(table_type='AGGREGATE', estimate_type="BE")

To query for rates three risk factors need to be provided: CalendarYear, Gender and Age. Two of them are encoded in the canonical way, to be sure we get the gender right we import the corresponding *RiskFactor class* from protolinc and then proceed creating three arrays with two entries each:

In [5]:
import numpy as np
from protolinc.models.risk_factors import Gender

ages = np.array([60, 70], dtype=np.int16)
genders = np.array([Gender.M, Gender.F], dtype=np.int16)
calendar_years = np.array([2022, 2022], dtype=np.int16)

These arrays parametrize two individuals in the year 2022: a 60 year old male and a 70 year old female. For
both of them we can now lookup the mortality rates as follows.

In [6]:
rates_provider.get_rates(age=ages, gender=genders, calendaryear=calendar_years)

array([0.00392406, 0.00597981])

### Using the DAV Table via Configuration

The key change when compared to the example 1 is found in the *longevity_assumptions.yml* file which reads as follows:

In [11]:
with open('longevity_assumptions.yml', 'r') as f:
    print(f.read())

# for the annuity model
assumptions_spec:

  be:
    # annuitant's deaths
    - [0, 1, ["DAV2004R", "table_type:AGGREGATE", "estimate_type:BE",
              "base_directory:tables/Germany_Annuities_DAV2004R"]]

  res:
    # annuitant's deaths
    - [0, 1, ["DAV2004R", "table_type:AGGREGATE", "estimate_type:LOADED",
              "base_directory:tables/Germany_Annuities_DAV2004R"]]




Note that the parametrization is not a simple *scalar* as in example 1 but uses a syntax which should be
easy to grasp with the knowledge of the above.

We can now immediately start a run from the sheel by typeing ```protolinc run``` in the main directory of this example. We can also start the run programmatically as follows.

In [12]:
from protolinc.main import get_config_from_file, project_cashflows
run_config = get_config_from_file('config.yml')
project_cashflows(run_config)

INFO - 2022-07-02 14:12:35,296 - protolinc.main - Mulistate run with config: {'model_name': 'GenericMultiState', 'years_to_simulate': 119, 'portfolio_path': 'portfolio/portfolio_annuity_small.xlsx', 'assumptions_path': 'longevity_assumptions.yml', 'steps_per_month': 1, 'state_model_name': 'AnnuityRunoffStates', 'timestep_duration': 0.08333333333333333, 'outfile': 'results/ncf_out_generic.csv', 'portfolio_cache': 'portfolio/portfolio_cache', 'profile_out_dir': '.', 'portfolio_chunk_size': 1024, 'use_multicore': True}
INFO - 2022-07-02 14:12:35,360 - protolinc.portfolio - Porfolio loaded from cache
INFO - 2022-07-02 14:12:35,361 - protolinc.portfolio - Portolio rows: 1
DEBUG - 2022-07-02 14:12:35,363 - protolinc.portfolio - Splitting portfolio for product ANNUITYINPAYMENT.
DEBUG - 2022-07-02 14:12:35,367 - protolinc.portfolio - Initializing portfolio from dataframe
INFO - 2022-07-02 14:12:35,373 - protolinc.main - Executions in single process for 1 units
INFO - 2022-07-02 14:12:35,374 - 